In [3]:
import json


In [4]:
CSQA_DATA = '../data/csqa'
DEV_PATH = CSQA_DATA + '/dev_rand_split.jsonl'
TRAIN_PATH = CSQA_DATA + '/train_rand_split.jsonl'
IH_QID_PATH = CSQA_DATA + '/inhouse_split_qids.txt'

DEV_PREDICTED_PATH = '../runs/csqa/dragon__csqa_ih_true_load__elr1e-5_dlr1e-3_b128_ufz2_e22_sd5__20230324_111607/dev_e0_preds.csv'
TEST_PREDICTED_PATH = '../runs/csqa/dragon__csqa_ih_true_load__elr1e-5_dlr1e-3_b128_ufz2_e22_sd5__20230324_111607/test_e0_preds.csv'
STATEMENT_PATH = CSQA_DATA + '/statements'
DEV_STATEMENT_PATH = STATEMENT_PATH + '/dev.statements.jsonl'
TRAIN_STATEMENT_PATH = STATEMENT_PATH + '/train.statements.jsonl'


In [5]:

def load_txt(path):
    data = []
    with open(path, 'r') as f:
        for line in f:
            line= line.strip('\n')
            data.append(line)
    return data

def load_jsonl(path):
    data = []
    with open(path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

def load_csv(path):
    data = []
    with open(path, 'r') as f:
        for line in f:
            
            line = line.strip('\n')
            line= line.split(',')
            data.append(line)
    return data

dev_data = load_jsonl(DEV_PATH)
dev_pred = load_csv(DEV_PREDICTED_PATH)
test_pred = load_csv(TEST_PREDICTED_PATH)
train_data = load_jsonl(TRAIN_PATH)
ih_split_qids = load_txt(IH_QID_PATH)



print(ih_split_qids[0])
print(train_data[0])
print(test_pred[0])
print(dev_data[0])
print(dev_data[0]['question']['stem'])
print(dev_pred[0])

2340e5fb43bc9a5c93da7fbd99d6a852
{'answerKey': 'A', 'id': '075e483d21c29a511267ef62bedc0461', 'question': {'question_concept': 'punishing', 'choices': [{'label': 'A', 'text': 'ignore'}, {'label': 'B', 'text': 'enforce'}, {'label': 'C', 'text': 'authoritarian'}, {'label': 'D', 'text': 'yell at'}, {'label': 'E', 'text': 'avoid'}], 'stem': 'The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?'}}
['4c1cb0e95b99f72d55c068ba0255c54d', 'A']
{'answerKey': 'A', 'id': '1afa02df02c908a558b4036e80242fac', 'question': {'question_concept': 'revolving door', 'choices': [{'label': 'A', 'text': 'bank'}, {'label': 'B', 'text': 'library'}, {'label': 'C', 'text': 'department store'}, {'label': 'D', 'text': 'mall'}, {'label': 'E', 'text': 'new york'}], 'stem': 'A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?'}}
A revolving door is convenient for two direction travel, but it als

In [6]:
# ih_split_qids stores the ids of train data that are inhouse split
def get_train_data_ids():
    train_data_ids = []
    for data in train_data:
        train_data_ids.append(data['id'])
    return train_data_ids

train_data_ids = get_train_data_ids()

#remove the ids of train data that are inhouse split
ih_split_test_qids = [qid for qid in train_data_ids if qid not in ih_split_qids]

print(len(ih_split_test_qids))



def get_tset_data():
    test_data = []
    for qid in ih_split_test_qids:
        for data in train_data:
            if data['id'] == qid:
                test_data.append(data)
    return test_data



test_data = get_tset_data()


1241


In [7]:
print(len(test_data))
print(test_data[0])

1241
{'answerKey': 'A', 'id': '4c1cb0e95b99f72d55c068ba0255c54d', 'question': {'question_concept': 'choker', 'choices': [{'label': 'A', 'text': 'jewelry store'}, {'label': 'B', 'text': 'neck'}, {'label': 'C', 'text': 'jewlery box'}, {'label': 'D', 'text': 'jewelry box'}, {'label': 'E', 'text': 'boutique'}], 'stem': 'To locate a choker not located in a jewelry box or boutique where would you go?'}}


In [12]:
#get wrong predicted questions for dev data or test data
def get_wrong_predicted_questions(data,predicted_data,humen_readable=False):
    wrong_predicted_questions = []
    for i in range(len(data)):
        if data[i]['answerKey'] != predicted_data[i][1]:
            new_data = {}
            
            if humen_readable:
               
                new_data["question"] = data[i]['question']['stem']
                choices_label = [choice['label'] for choice in data[i]['question']['choices']]
                choices_text = [choice['text'] for choice in data[i]['question']['choices']]
                new_data["choices"] = {choices_label[i]:choices_text[i] for i in range(len(choices_label))}
                new_data["answerKey"] = data[i]['answerKey']

            else:
                new_data= data[i]
                # new_data["choices"] = data[i]['question']['choices']

            new_data["wrong_prediction"] = predicted_data[i][1]
            wrong_predicted_questions.append(new_data)
    return wrong_predicted_questions
    


# wrong_predicted_questions_test = get_wrong_predicted_questions(test_data,test_pred,humen_readable=True)
# print(len(wrong_predicted_questions_test))
# print(wrong_predicted_questions_test[0])
# wrong_predicted_questions_dev = get_wrong_predicted_questions(dev_data,dev_pred,humen_readable=True)
# print(len(wrong_predicted_questions_dev))
# print(wrong_predicted_questions_dev[0])

In [16]:
# write wrong predicted questions to json file
def write_jsonl(path,data):
    with open(path, 'w') as f:
        for line in data:
            f.write(json.dumps(line))
            f.write('\n')
    return

wrong_predicted_questions_dev= get_wrong_predicted_questions(dev_data,dev_pred)
wrong_predicted_questions_test = get_wrong_predicted_questions(test_data,test_pred)
wrong_predicted_questions_humen_dev = get_wrong_predicted_questions(dev_data,dev_pred,humen_readable=True)
wrong_predicted_questions_humen_test = get_wrong_predicted_questions(test_data,test_pred,humen_readable=True)

wrong_predicted_questions_humen_dev_path = 'humen_readable_wrong_predicted_questions_dev.jsonl'
wrong_predicted_questions_humen_test_path = 'humen_readable_wrong_predicted_questions_test.jsonl'
wrong_predicted_questions_dev_path = 'wrong_predicted_questions_dev.jsonl'
wrong_predicted_questions_test_path= 'wrong_predicted_questions_test.jsonl'

write_jsonl(wrong_predicted_questions_humen_dev_path,wrong_predicted_questions_humen_dev)
write_jsonl(wrong_predicted_questions_humen_test_path,wrong_predicted_questions_humen_test)
write_jsonl(wrong_predicted_questions_dev_path,wrong_predicted_questions_dev)
write_jsonl(wrong_predicted_questions_test_path,wrong_predicted_questions_test)